In [78]:


import glob 
svpath = "SavedResults/"
resultStrings = glob.glob(svpath+"savedBestSeed-21*")
resultStrings2 = glob.glob(svpath+"savedBestSeed-22*")
resultStrings = resultStrings + resultStrings2
#resultStrings = glob.glob("F:/PythonProjects/NietoExcercise-1/SavedResults/TestFold/savedBestSeed-5*")
#resultStrings = glob.glob("F:/PythonProjects/NietoExcercise-1/SavedResultsNN/sa*")
#skipsubj = True
skipSubjects = []
onlySignificant = True
minAcc = 0.9



In [81]:
!pip3 install tabulate -q
!pip3 install prettytable -q

from array import ArrayType
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in resultStrings:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10 
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s , subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)
    
    currSeed = subject[1]
    
    currSubj = subject[2]


    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] +=1


    for aResults in subject[3]: 
        for aResults2 in aResults:
            #If the np array comes from svm results
            if len(aResults2)<6:
                
                aName = aResults2[0]
                ares = aResults2[1]
               
            
                if type(ares) is tuple:
                    ares = ares[0]
                    
                akernel = aResults2[2]
                aC = round(aResults2[3],2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1
        
                combNr = combinationList.index([nameDict[aName], kernelDict[akernel], aC])
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    index = int(np.where(sublist[:,0] == combNr)[0])
                    
                    combinationResult[currSubj][index][1] = combinationResult[currSubj][index][1] * (1-(1/nrOfSeedsPerSubject[currSubj])) + ares*(1/nrOfSeedsPerSubject[currSubj])
                    
                else:
                  
                    combinationResult[currSubj].append([combNr, ares]) 
                    
                    #If the np array comes from NN results
            else:
               
                aName = aResults2[0]
                ares = aResults2[1]
 
                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3],2)
                areg = round(aResults2[4],2)
                adrp = round(aResults2[5],2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aLz, areg, adrp] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aLz, areg, adrp])
                    nrOfCombination += 1
                #combResult = []
                combNr = combinationList.index([nameDict[aName], kernelDict[akernel], aLz, areg, adrp])
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:,0] == combNr)[0])
                    
                    combinationResult[currSubj][index][1] = combinationResult[currSubj][index][1] * (1-(1/nrOfSeedsPerSubject[currSubj])) + ares*(1/nrOfSeedsPerSubject[currSubj])
                    
                else:
                    
                    combinationResult[currSubj].append([combNr, ares])



#Making table

#headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination],dtype=object) 
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination) 

for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] +  res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] +  res[1]
            nrOfSubjects[res[0]] +=1
            subjectResultArray[subNr, res[0]] = res[1]

#Print the table!
import pandas as pd
comboAverage = comboAverage/nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option('display.max_columns', None)


subjectResultArray2 = np.array([np.where(subjectResultArray != None)[0], subjectResultArray[np.where(subjectResultArray != None)]])

headers2 = headers[np.where(np.array(subjectResultArray) != None)[1]]
table = pd.DataFrame(np.array([subjectResultArray2[1]]) ,columns = headers2)
print("sad")
print(subjectResultArray2.shape)
display(table)
if subjectResultArray2.shape[1] < 20:
    for header in headers2:
        ind = int(header)
        name = list(nameDict.keys())[combinationList[ind][0]]
        kernel = list(kernelDict.keys())[combinationList[ind][1]]
        C = combinationList[ind][2]
        acc = round(subjectResultArray[1 , ind],2)
        print(f"header: {int(header)} name: {name}, kernel: {kernel} C: {C} , Accuracy {acc}")



(1, 165)
(10, 165)
sad
(2, 8)


c:\Users\Luna\anaconda3\envs\mentalab3\lib\site-packages\ipykernel_launcher.py:142: RuntimeWarning: invalid value encountered in true_divide


,9,10,20,21,31,32,130,131
0,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9


header: 9 name: dataWCV, kernel: sigmoid C: 3.88 , Accuracy 0.9
header: 10 name: dataWCV, kernel: sigmoid C: 5.0 , Accuracy 0.9
header: 20 name: dataWCV2, kernel: sigmoid C: 3.88 , Accuracy 0.9
header: 21 name: dataWCV2, kernel: sigmoid C: 5.0 , Accuracy 0.9
header: 31 name: dataWCVdataWCV2, kernel: sigmoid C: 3.88 , Accuracy 0.9
header: 32 name: dataWCVdataWCV2, kernel: sigmoid C: 5.0 , Accuracy 0.9
header: 130 name: dataWCVdataCV2, kernel: sigmoid C: 3.88 , Accuracy 0.9
header: 131 name: dataWCVdataCV2, kernel: sigmoid C: 5.0 , Accuracy 0.9


In [ ]:
print(subjectResultArray2[1])

[0.6 0.5 0.8500000000000001 0.8500000000000001 0.9000000000000001
 0.9000000000000001 0.6 0.5 0.85 0.85 0.9 0.9 0.6 0.5 0.85 0.85 0.9 0.9
 0.8 0.65 0.75 0.8 0.75 0.75 0.55 0.8 0.75 0.65 0.6 0.8000000000000002
 0.6500000000000001 0.7500000000000001 0.7500000000000001
 0.7500000000000001 0.7500000000000001 0.8500000000000001
 0.7000000000000001 0.6 0.6 0.8 0.7 0.7 0.75 0.75 0.8 0.7 0.55 0.6 0.55
 0.55 0.8 0.65 0.75 0.75 0.75 0.75 0.5 0.8 0.7 0.7 0.55 0.75 0.6 0.7 0.75
 0.75 0.75 0.75 0.75 0.6 0.6 0.6 0.8 0.65 0.7 0.75 0.75 0.75 0.65 0.75 0.6
 0.55 0.5 0.75 0.65 0.7 0.8 0.75 0.75 0.7 0.8 0.65 0.55 0.5
 0.7500000000000001 0.7000000000000001 0.7000000000000001
 0.7000000000000001 0.7500000000000001 0.7500000000000001
 0.7000000000000001 0.6 0.8000000000000002 0.7000000000000001
 0.7000000000000001 0.8000000000000002 0.8000000000000002
 0.7000000000000001 0.8500000000000001 0.9000000000000001
 0.9000000000000001 0.7500000000000001 0.7000000000000001
 0.7000000000000001 0.7000000000000001 0.7

In [ ]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option('display.max_columns', None)
table = pd.DataFrame(np.array(comboAverage) ,columns = headers)
display(table)


print(f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}")
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(maxInd)
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")

[0.         0.71388889 0.57327044 0.52777778 0.54957806 0.53676471
 0.56704545 0.53425926 0.54473684 0.53472222]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164
0,0.58206,0.5,0.547222,0.559848,0.566098,0.556023,NaN,0.645833,0.659722,0.765625,0.78125,0.593981,0.5,NaN,NaN,NaN,NaN,NaN,0.7625,0.7625,0.810417,0.820833,0.593981,0.5,NaN,NaN,NaN,NaN,NaN,0.7625,0.7625,0.810417,0.820833,0.575,0.65,0.75,0.670833,0.645833,0.6875,0.55,0.8,0.652778,0.575,0.533333,0.621759,0.65,0.566667,0.580988,0.578463,0.597854,0.5,0.662963,0.556667,0.558333,0.55625,0.6,0.6,0.620833,0.597222,0.652778,0.683333,0.7,0.525,0.55,0.525,0.5125,0.585,0.575,0.75,0.666667,0.583333,0.625,0.5,0.65,0.6,0.574306,0.533333,0.597222,0.55,0.580556,0.611111,0.625,0.638889,0.75,0.625,0.533333,0.533333,0.547222,0.691667,0.65,0.620833,0.666667,0.625,0.638889,0.595833,0.75,0.6,0.544444,0.520833,0.597222,0.65,0.6,0.691667,0.6875,0.6875,0.7,0.8,0.65,0.535185,0.518519,0.556818,0.581818,0.572727,0.57178,0.584091,0.592424,0.580556,0.588889,0.527778,0.5,0.530556,0.618333,NaN,0.57178,0.59697,0.578409,0.594091,0.5,0.627778,0.5875,0.741667,0.633333,0.585227,0.6,0.592273,0.549242,0.58428,0.594697,0.7,0.589583,0.572917,0.575,0.552083,0.612626,0.6,0.627525,0.593939,0.645606,0.645606,0.75,0.8,0.535417,0.516667,0.516667,0.612626,0.6,0.612121,0.641414,0.628977,0.601515,0.65,0.648485,0.575,0.6,0.533333


Max average accuracy 0.8208333333333333
21
dataWCV2
sigmoid
C: 5.0


In [ ]:
table = pd.DataFrame(np.array(comboTotal) ,columns = headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal)==False)]))
maxInd2 = int(np.where(comboTotal == maxVal2)[1])
print(maxInd2)
print(nrOfSubjects[maxInd2])
print(f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} ")

print(list(nameDict.keys())[combinationList[maxInd2][0]])
print(list(kernelDict.keys())[combinationList[maxInd2][1]])
print(f"C: {combinationList[maxInd2][2]} ")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164
0,3.492361,0.5,1.641667,2.799242,2.830492,2.780114,0.0,1.9375,1.979167,1.53125,1.5625,1.781944,0.5,0.0,0.0,0.0,0.0,0.0,1.525,1.525,1.620833,1.641667,1.781944,0.5,0.0,0.0,0.0,0.0,0.0,1.525,1.525,1.620833,1.641667,2.3,0.65,0.75,1.341667,1.291667,1.375,0.55,0.8,1.305556,1.15,1.6,3.730556,0.65,2.833333,4.066919,4.049242,3.587121,0.5,1.988889,2.783333,2.233333,2.225,1.8,1.2,1.241667,1.791667,1.958333,2.05,0.7,1.05,1.1,1.05,2.05,2.925,1.15,0.75,1.333333,2.333333,1.875,0.5,1.3,1.2,2.297222,2.133333,1.791667,1.1,1.741667,1.833333,1.875,1.916667,0.75,1.25,1.6,1.6,1.641667,1.383333,0.65,1.241667,1.333333,1.875,1.916667,1.191667,0.75,0.6,1.633333,2.083333,1.791667,0.65,1.2,1.383333,1.375,1.375,0.7,0.8,0.65,1.605556,1.555556,3.340909,1.745455,2.290909,2.287121,2.920455,2.962121,1.741667,1.766667,1.583333,1.5,1.591667,3.091667,0.0,2.287121,1.790909,3.470455,2.970455,0.5,1.255556,2.35,1.483333,1.9,2.340909,1.2,2.961364,3.295455,2.337121,2.378788,0.7,2.358333,2.291667,2.875,2.208333,3.063131,1.2,1.882576,2.969697,3.22803,3.22803,0.75,0.8,2.141667,1.55,1.55,3.063131,0.6,1.836364,1.924242,2.515909,3.007576,0.65,1.945455,2.3,1.8,1.6


47
7.0
Max total significant accuracy then averaged 0.580988455988456 
dataCV2
rbf
C: 2.75 


In [ ]:


maxAcc = np.max([res for res in np.array(subjectResultArray[subjectResultArray!=None])])
print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
print(np.max([res for res in np.array(subjectResultArray[subjectResultArray!=None])]))

Max single accuracy position (array([1, 1, 1, 1], dtype=int64), array([  9,  10, 130, 131], dtype=int64))
0.9000000000000001
